In [1]:
from skt.ye import get_spark
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pyspark.sql.functions as F
import sys
from datetime import datetime, timedelta

In [2]:
from skt.gcp import load_bigquery_ipython_magic, \
                    bq_to_pandas, \
                    get_bigquery_client, \
                    bq_to_df

In [36]:
execution_dt_datetime = datetime.strptime(current_dt, '%Y%m%d')
execution_dt = (execution_dt_datetime - timedelta(days=1)).strftime('%Y-%m-%d')
execution_dt_luna = (execution_dt_datetime - timedelta(days=2)).strftime('%Y-%m-%d')
print(f'excution_dt: {execution_dt}')
print(f'excution_dt_luna: {execution_dt_luna}')

excution_dt: 2023-12-09
excution_dt_luna: 2023-12-08


In [9]:
user_id = "x1112436"
delete_table1 = f"adot_musichome_app_log_stg"

In [13]:
query = f"""
SELECT DISTINCT dt
FROM `{user_id}.{delete_table1}`
"""

In [14]:
partition_list_table1 = bq_to_pandas(query)

unsupported operand type(s) for /: 'NoneType' and 'int'
Downloading: 100%|██████████|
query: 
SELECT DISTINCT dt
FROM `x1112436.adot_musichome_log_stg`

destination: skt-datahub._775c5ccab1096b3cccd7ac34a5db11c0a354fb07.anonb6dcc8029485aafcb2ba355cee3b3138d3fdcf9263d8a34ce3121474b731976e
total_rows: 45
slot_secs: 0.823

Downloading: 100%|██████████|


In [37]:
partition_list = list(partition_list_table1.dt.unique())

# JOIN AND REWRITEfrom skt.gcp import pandas_to_bq, get_bigquery_client

In [28]:
from skt.gcp import pandas_to_bq, get_bigquery_client

In [30]:
for dt in partition_list:
    query_after = f"""
        SELECT A.*
            FROM (
            SELECT *
            FROM `{user_id}.{delete_table1}`
            WHERE dt = '{dt}'
        ) A
        INNER JOIN
        (
            SELECT DISTINCT luna_id as user_id 
            FROM apollo.luna_user 
            WHERE coalesce(status,'')!='CLOSED' and dt=(select max(dt) from apollo.luna_user)
        ) B
        ON A.USER_ID = B.USER_ID
    """
    query_before = f"""
         SELECT *
         FROM `{user_id}.{delete_table1}`
         WHERE dt = '{dt}'
    """
    
    valid_user_musichome_app_log = bq_to_pandas(query)
    before_user = bq_to_pandas(query2)
    
    diff = len(before_user) - len(valid_user_musichome_app_log)
    if diff > 0:
        print(f'dt: {dt}')
        print(f'before: {len(before_user)}')
        print(f'after: {len(valid_user_musichome_app_log)}')
        print(f'diff log: {diff}')
        
    elif diff < 0:
        raise Exception 
    
    else:
        continue

    pandas_to_bq(
             pd_df = valid_user_musichome_app_log, 
             destination = f"skt-datahub.{user_id}.{delete_table1}", 
             partition = 'dt',
             overwrite=True
    )

query: 
        SELECT A.*
            FROM (
            SELECT *
            FROM `x1112436.adot_musichome_app_log_stg`
            WHERE dt = '2023-10-31'
        ) A
        INNER JOIN
        (
            SELECT DISTINCT luna_id as user_id 
            FROM apollo.luna_user 
            WHERE coalesce(status,'')!='CLOSED' and dt=(select max(dt) from apollo.luna_user)
        ) B
        ON A.USER_ID = B.USER_ID
    
destination: skt-datahub._775c5ccab1096b3cccd7ac34a5db11c0a354fb07.anon842e09e84c27ad27f9d0bafd0de74083c33b1797172bfad8bb7ef0eff9fc6693
total_rows: 26195
slot_secs: 5.22

Downloading: 100%|██████████|
query: 
         SELECT *
         FROM `x1112436.adot_musichome_app_log_stg`
         WHERE dt = '2023-10-31'
    
destination: skt-datahub._775c5ccab1096b3cccd7ac34a5db11c0a354fb07.anon3e4e1b9deb28d92da517637d4b358cdd51c18d3d0345801745f8c4a66ac1561d
total_rows: 26195
slot_secs: 0.412

Downloading: 100%|██████████|
dt: 2023-10-31
before: 26195
after: 26195
diff log: 0
q